In [81]:
import random

from sc2 import maps
from sc2.ids.unit_typeid import UnitTypeId
from sc2.player import Bot, Computer
from sc2.main import run_game
from sc2.data import Race, Difficulty
from sc2.bot_ai import BotAI
import nest_asyncio
nest_asyncio.apply()

In [82]:
# self.minerals: int
# self.vespene: int
# self.supply_army: int # 0 at game start
# self.supply_workers: int # 12 at game start
# self.supply_cap: int # 14 for zerg, 15 for T and P at game start
# self.supply_used: int # 12 at game start
# self.supply_left: int # 2 for zerg, 3 for T and P at game start

class WorkerRushBot(BotAI):
    async def on_step(self, iteration: int):

        print(f"iter:{iteration},minerals:{self.minerals},"
              f"vespene:{self.vespene},supply_army:{self.supply_army},"
              f"supply_workers:{self.supply_workers},supply_cap:{self.supply_cap},"
              f"supply_used:{self.supply_used},supply_left:{self.supply_left},",
              f"idle_worker_count:{self.idle_worker_count},probes:{self.units(UnitTypeId.PROBE).amount},"
              f"pylons:{self.structures(UnitTypeId.PYLON).amount},forges:{self.structures(UnitTypeId.FORGE).amount},"
              f"cannons:{self.structures(UnitTypeId.PHOTONCANNON).amount},voidarys:{self.units(UnitTypeId.VOIDRAY).amount},")

        #市政厅
        if self.townhalls:
            nexus = self.townhalls.random
            await self.distribute_workers()
            #虚空光舰
            if self.units(UnitTypeId.VOIDRAY).amount<150:
                for sg in self.structures(UnitTypeId.STARGATE).ready.idle:
                    if self.can_afford(UnitTypeId.VOIDRAY):
                        sg.train(UnitTypeId.VOIDRAY)

            #探机
            if self.can_afford(UnitTypeId.PROBE) and nexus.is_idle and self.units(UnitTypeId.PROBE).amount<25:
                nexus.train(UnitTypeId.PROBE)
            #水晶塔
            elif not self.structures(UnitTypeId.PYLON) and self.already_pending(UnitTypeId.PYLON)==0:
                if self.can_afford(UnitTypeId.PYLON):
                    await self.build(UnitTypeId.PYLON,near=nexus)
            elif self.structures(UnitTypeId.PYLON).amount < 5:
                if self.can_afford(UnitTypeId.PYLON):
                  target_pylon = self.structures(UnitTypeId.PYLON).closest_to(self.enemy_start_locations[0])
                  pos = target_pylon.position.towards(self.enemy_start_locations[0],random.randrange(8,15))
                  await self.build(UnitTypeId.PYLON,near=pos)
            #吸收间
            elif self.structures(UnitTypeId.ASSIMILATOR).amount <= 1:
                for vespene in self.vespene_geyser.closer_than(15,nexus):
                    if self.can_afford(UnitTypeId.ASSIMILATOR):
                        await self.build(UnitTypeId.ASSIMILATOR,near=vespene)

            #熔炉
            elif not self.structures(UnitTypeId.FORGE) and self.already_pending(UnitTypeId.FORGE)==0:
                if self.can_afford(UnitTypeId.FORGE):
                    await self.build(UnitTypeId.FORGE,near=self.structures(UnitTypeId.PYLON).closest_to(nexus))
            #光子炮
            elif self.structures(UnitTypeId.FORGE).ready and self.structures(UnitTypeId.PHOTONCANNON).amount < 3:
                if self.can_afford(UnitTypeId.PHOTONCANNON):
                    await self.build(UnitTypeId.PHOTONCANNON,near=nexus)
            #传送门
            elif not self.structures(UnitTypeId.GATEWAY) and self.already_pending(UnitTypeId.GATEWAY)==0:
                if self.can_afford(UnitTypeId.GATEWAY):
                    await self.build(UnitTypeId.GATEWAY,near=self.structures(UnitTypeId.PYLON).closest_to(nexus))
            #控制核心
            elif not self.structures(UnitTypeId.CYBERNETICSCORE) and self.already_pending(UnitTypeId.CYBERNETICSCORE)==0:
                if self.can_afford(UnitTypeId.CYBERNETICSCORE):
                    await self.build(UnitTypeId.CYBERNETICSCORE,near=self.structures(UnitTypeId.PYLON).closest_to(nexus))

            #星门
            elif not self.structures(UnitTypeId.STARGATE):
                if self.can_afford(UnitTypeId.STARGATE):
                    await self.build(UnitTypeId.STARGATE,near=self.structures(UnitTypeId.PYLON).closest_to(nexus))

        else:
            if self.can_afford(UnitTypeId.NEXUS):
                await self.expand_now()
        #发动进攻
        if self.units(UnitTypeId.VOIDRAY).amount > 5:
            if self.enemy_units:
                for vg in self.units(UnitTypeId.VOIDRAY).idle:
                    vg.attack(random.choice(self.enemy_units))
            elif self.enemy_structures:
                for vg in self.units(UnitTypeId.VOIDRAY).idle:
                    vg.attack(random.choice(self.enemy_structures))
            else:
                for vg in self.units(UnitTypeId.VOIDRAY).idle:
                    vg.attack(self.enemy_start_locations[0])

In [83]:
run_game(maps.get("WorldofSleepersLE"), [
    Bot(Race.Protoss, WorkerRushBot()),
    Computer(Race.Random, Difficulty.Hard)
], realtime=False)


2022-08-21 21:43:45.166 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.launched (was None)
2022-08-21 21:43:45.166 | INFO     | sc2.controller:create_game:37 - Creating new game
2022-08-21 21:43:45.167 | INFO     | sc2.controller:create_game:38 - Map:     WorldofSleepersLE
2022-08-21 21:43:45.167 | INFO     | sc2.controller:create_game:39 - Players: Bot WorkerRushBot(Protoss), Computer Hard(Random, RandomBuild)
2022-08-21 21:43:45.169 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.init_game (was Status.launched)
2022-08-21 21:43:50.977 | INFO     | sc2.protocol:_execute:72 - Client status changed to Status.in_game (was None)
2022-08-21 21:43:50.977 | INFO     | sc2.main:_play_game:221 - Player 1 - Bot WorkerRushBot(Protoss)
iter:0,minerals:50,vespene:0,supply_army:0,supply_workers:12,supply_cap:15,supply_used:12,supply_left:3, idle_worker_count:0,probes:12,pylons:0,forges:0,cannons:0,voidarys:0,
iter:1,minerals:0,vespene:0,supply_army:0

<Result.Victory: 1>